In [ ]:
import os
import sys
import time

import GCode
import GRBL
import matplotlib.pyplot as plt
import numpy as np

# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

"Napkin" scratches.

![](http://luckofthedraw.fun/.imgs/brick_0003.jpeg)

Drawn by hand. ~18mm brick height.

![](http://luckofthedraw.fun/.imgs/brick_0001.jpeg)

> Standard bricks. The standard co-ordinating size for brickwork is 225 mm x 112.5 mm x 75 mm (length x depth x height). This includes 10 mm mortar joints, and so the standard size for a brick itself is 215 mm x 102.5 mm x 65 mm (length x depth x height).

In [2]:
# Standard brick dimensions.
BrickHeight = 65  # [mm]
BrickLength = 225  # [mm]
BrickDepth = 12.5  # [mm]
BrickRatio = 215 / 65  # [dimensionless]


In [3]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 2 * BlockHeight  # mm


In [4]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 5  # [dimensionless]

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows  # [mm]
L_Block_Brick = H_Block_Brick * BrickRatio  # [mm]


# Code:

In [5]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    np.eye(2),  # Do nothing, for debugging.
    flip,  # Flip the matrix, reduces travel time.
)
vertical_brick_lines_tuple = (
    np.arange(L_Block_Brick, BlockLength, L_Block_Brick),  # Odd rows.
    np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick),  # Even rows.
)
horizontal_brick_lines = np.linspace(
    0, BlockHeight, N_BrickRows, endpoint=False
)


Lines parallel to the X-axis.
Separates rows of bricks.

In [6]:
horizontal_brick_lines


array([ 0. , 17.8, 35.6, 53.4, 71.2])

Lines parallel to the Y-axis.

In [7]:
vertical_brick_lines_tuple


(array([ 58.87692308, 117.75384615, 176.63076923]),
 array([ 29.43846154,  88.31538462, 147.19230769]))

In [8]:
BlockBrick = GCode.Program()
BlockBrick


Program<lines=0>

In [9]:
BlockBrick.lines


[]

In [10]:
BlockBrick.lines = list()

for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )
    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]

    row_line_points = np.matmul(transform, row_line_points)
    line_ = GCode.Line(points=row_line_points)

    BlockBrick.lines.append(line_)

    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[
        np.mod(idx, len(vertical_brick_lines_tuple))
    ]
    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]
    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]
        column_line_points = (
            np.array(
                [
                    [start_point_y, vertical_brick_line],
                    [end_point_y, vertical_brick_line],
                ]
            ),
        )
        column_line_points = np.matmul(transform, column_line_points)
        line_ = GCode.Line(column_line_points)
        BlockBrick.lines.append(line_)


In [11]:
BlockBrick


Program<lines=16>

In [12]:
BlockBrick.lines


[Line<cut=178.0mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=178.0mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=17.8mm, F=300, S=150>,
 Line<cut=178.0mm, F=300, S=150>,
 Line<cut=17.800000000000004mm, F=300, S=150>,
 Line<cut=17.800000000000004mm, F=300, S=150>,
 Line<cut=17.800000000000004mm, F=300, S=150>,
 Line<cut=178.0mm, F=300, S=150>,
 Line<cut=17.799999999999997mm, F=300, S=150>,
 Line<cut=17.799999999999997mm, F=300, S=150>,
 Line<cut=17.799999999999997mm, F=300, S=150>]

In [13]:
BlockBrick.lines[0]


Line<cut=178.0mm, F=300, S=150>

In [14]:
BlockBrick.lines[1]


Line<cut=17.8mm, F=300, S=150>

In [15]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()


(25, ['', "Grbl 1.1f ['$' for help]", "[MSG:'$H'|'$X' to unlock]"])

In [16]:
cnc.home()


14

In [18]:
BlockBrick.generate_gcode()


In [19]:
BlockBrick.buffer


['G28',
 'G0 F120',
 'M3 S1',
 'G21',
 'G90',
 'G92 X0 Y0 Z0',
 'G0 X0.0 Y17.8',
 'M4 S150',
 'G1 X178.0 Y17.8 F300',
 'M5',
 'G0 X0.0 Y29.4385',
 'M4 S150',
 'G1 X17.8 Y29.4385 F300',
 'M5',
 'G0 X0.0 Y88.3154',
 'M4 S150',
 'G1 X17.8 Y88.3154 F300',
 'M5',
 'G0 X0.0 Y147.1923',
 'M4 S150',
 'G1 X17.8 Y147.1923 F300',
 'M5',
 'G0 X0.0 Y35.6',
 'M4 S150',
 'G1 X178.0 Y35.6 F300',
 'M5',
 'G0 X17.8 Y58.8769',
 'M4 S150',
 'G1 X35.6 Y58.8769 F300',
 'M5',
 'G0 X17.8 Y117.7538',
 'M4 S150',
 'G1 X35.6 Y117.7538 F300',
 'M5',
 'G0 X17.8 Y176.6308',
 'M4 S150',
 'G1 X35.6 Y176.6308 F300',
 'M5',
 'G0 X0.0 Y53.4',
 'M4 S150',
 'G1 X178.0 Y53.4 F300',
 'M5',
 'G0 X35.6 Y29.4385',
 'M4 S150',
 'G1 X53.4 Y29.4385 F300',
 'M5',
 'G0 X35.6 Y88.3154',
 'M4 S150',
 'G1 X53.4 Y88.3154 F300',
 'M5',
 'G0 X35.6 Y147.1923',
 'M4 S150',
 'G1 X53.4 Y147.1923 F300',
 'M5',
 'G0 X0.0 Y71.2',
 'M4 S150',
 'G1 X178.0 Y71.2 F300',
 'M5',
 'G0 X53.4 Y58.8769',
 'M4 S150',
 'G1 X71.2 Y58.8769 F300',
 'M5',
 'G0

In [20]:
BlockBrick.generate_gcode()


In [21]:
BlockBrick.buffer


['G28',
 'G0 F120',
 'M3 S1',
 'G21',
 'G90',
 'G92 X0 Y0 Z0',
 'G0 X0.0 Y17.8',
 'M4 S150',
 'G1 X178.0 Y17.8 F300',
 'M5',
 'G0 X0.0 Y29.4385',
 'M4 S150',
 'G1 X17.8 Y29.4385 F300',
 'M5',
 'G0 X0.0 Y88.3154',
 'M4 S150',
 'G1 X17.8 Y88.3154 F300',
 'M5',
 'G0 X0.0 Y147.1923',
 'M4 S150',
 'G1 X17.8 Y147.1923 F300',
 'M5',
 'G0 X0.0 Y35.6',
 'M4 S150',
 'G1 X178.0 Y35.6 F300',
 'M5',
 'G0 X17.8 Y58.8769',
 'M4 S150',
 'G1 X35.6 Y58.8769 F300',
 'M5',
 'G0 X17.8 Y117.7538',
 'M4 S150',
 'G1 X35.6 Y117.7538 F300',
 'M5',
 'G0 X17.8 Y176.6308',
 'M4 S150',
 'G1 X35.6 Y176.6308 F300',
 'M5',
 'G0 X0.0 Y53.4',
 'M4 S150',
 'G1 X178.0 Y53.4 F300',
 'M5',
 'G0 X35.6 Y29.4385',
 'M4 S150',
 'G1 X53.4 Y29.4385 F300',
 'M5',
 'G0 X35.6 Y88.3154',
 'M4 S150',
 'G1 X53.4 Y88.3154 F300',
 'M5',
 'G0 X35.6 Y147.1923',
 'M4 S150',
 'G1 X53.4 Y147.1923 F300',
 'M5',
 'G0 X0.0 Y71.2',
 'M4 S150',
 'G1 X178.0 Y71.2 F300',
 'M5',
 'G0 X53.4 Y58.8769',
 'M4 S150',
 'G1 X71.2 Y58.8769 F300',
 'M5',
 'G0

In [23]:
cnc.status


'<Idle|MPos:-259.000,-174.000,-1.000|Bf:15,127|FS:0,0|Ov:100,100,100>'

In [24]:
cnc.cmd("$C")


['ok', '[MSG:Enabled]', 'ok']

In [25]:
cnc.cmd("$C")


['ok', '[MSG:Disabled]', 'ok', '', "Grbl 1.1f ['$' for help]"]

In [33]:
cnc.reset()


(0, ['ok', '', "Grbl 1.1f ['$' for help]", "[MSG:'$H'|'$X' to unlock]", 'ok'])

In [34]:
cnc.home()


12

In [37]:
for line in BlockBrick.buffer:
    ret = cnc.cmd(line)


In [45]:
cnc.cmd("")


[]

In [26]:
    time.sleep(0.5)
    try:
        assert(len(ret) == 2)
        assert(ret[0]=="ok")
        assert(ret[1]=="ok")
    except:
        print(line)
        cnc.reset()
        cnc.home()
        cnc.cmd("G28")
        break

M3 S1


In [29]:
cnc.status


'<Alarm|MPos:-257.775,-158.979,-1.000|Bf:15,127|FS:0,0|Ov:100,100,100>'

In [ ]:
BlockBrick.buffer
